# 客户端

这是一个与具有“认证(auth)”功能的服务器交互的示例客户端。

请参阅服务器代码和 FastAPI 中的相关文档，以确保实际安全性。

## 作为 Alice 登录

In [1]:
import requests

response = requests.post("http://localhost:8000/token", data={"username": "alice", "password": "secret1"})
result = response.json()

In [2]:
token = result['access_token']

In [3]:
inputs = {"input": "hello"}
response = requests.post("http://localhost:8000/invoke", 
    json={
        'input': '你好',
    },
    headers={
        'Authorization': f"Bearer {token}"
    }
)


In [4]:
response.json()

{'output': [{'id': None,
   'metadata': {'owner_id': 'alice'},
   'page_content': '猫喜欢老鼠',
   'type': 'Document'},
  {'id': None,
   'metadata': {'owner_id': 'alice'},
   'page_content': '猫喜欢奶酪',
   'type': 'Document'}],
 'metadata': {'run_id': '869032e5-0635-4a16-9fa8-1c00c9d88e73',
  'feedback_tokens': []}}

您也可以通过 RemoteRunnable 接口与之交互（用于其他链中）

In [5]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/", headers={"Authorization": f"Bearer {token}"})

In [6]:
await remote_runnable.ainvoke("cat")

[Document(metadata={'owner_id': 'alice'}, page_content='猫喜欢老鼠'),
 Document(metadata={'owner_id': 'alice'}, page_content='猫喜欢奶酪')]

## 作为 John 登录

In [7]:
import requests

response = requests.post("http://localhost:8000/token", data={"username": "john", "password": "secret2"})
token = response.json()['access_token']
remote_runnable = RemoteRunnable("http://localhost:8000/", headers={"Authorization": f"Bearer {token}"})

In [12]:
await remote_runnable.ainvoke("water")

[Document(metadata={'owner_id': 'john'}, page_content='我喜欢海边散步'),
 Document(metadata={'owner_id': 'john'}, page_content='狗喜欢棍子'),
 Document(metadata={'owner_id': 'john'}, page_content='我最喜欢的食物是奶酪')]